In [2]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import hdbscan
import json
from tqdm import tqdm

# Première piste (-pas retenue-)

In [3]:
'''import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import hdbscan
from tqdm import tqdm
import json

def load_full_dataset(file_path, use_columns=None, sample_frac=None):
    try:
        print("Chargement des données...")
        df = pd.read_csv(file_path, usecols=use_columns)

        # Échantillonnage aléatoire + reset de l'index
        if sample_frac is not None and 0 < sample_frac < 1:
            df = df.sample(frac=sample_frac).reset_index(drop=True)

        # Convertir la colonne 'embedding' (string JSON) en array numpy
        df['embedding'] = df['embedding'].apply(lambda x: np.array(json.loads(x)))

        print(f"Chargement terminé. Nombre de lignes: {len(df)}")
        return df
    except Exception as e:
        print(f"Erreur lors du chargement du dataset : {e}")
        return pd.DataFrame()


def apply_kmeans(embeddings, n_clusters=150):
    """
    Applique KMeans et renvoie (labels, modèle_kmeans).
    """
    print("Application de KMeans...")
    print(f"Nombre de points: {len(embeddings)}")

    # kmeans : n_init='auto' dans les versions récentes de sklearn
    kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init='auto')
    labels = kmeans.fit_predict(embeddings)

    print("KMeans terminé. Distribution des clusters :")
    unique, counts = np.unique(labels, return_counts=True)
    for u, c in zip(unique, counts):
        print(f" - Cluster KMeans {u}: {c} points")

    return labels, kmeans

def refine_with_hdbscan_per_kmeans_cluster(df, min_cluster_size=5, min_samples=3):
    """
    Pour chaque cluster KMeans, applique HDBSCAN afin de détecter
    des sous-clusters plus fins. Les labels finaux sont stockés
    dans la colonne hdbscan_cluster.
    """
    print("\nDébut du raffinement avec HDBSCAN pour chaque cluster KMeans...")

    # Tableau de labels HDBSCAN initialisé à -1 (bruit)
    hdbscan_labels = np.full(len(df), -1, dtype=int)
    
    # Ce décalage permet de numéroter de manière unique tous les sous-clusters,
    # même s'ils proviennent de différents clusters KMeans.
    global_offset = 0

    total_clusters_found = 0
    total_points_clustered = 0

    for cluster_id in tqdm(df['kmeans_cluster'].unique(), desc="Clusters KMeans"):
        # Extraire les données appartenant au cluster KMeans courant
        cluster_data = df[df['kmeans_cluster'] == cluster_id].copy()
        n_points = len(cluster_data)

        print(f"\nTraitement du cluster KMeans {cluster_id} contenant {n_points} points.")

        # Skip si trop peu de points
        if n_points < 3:
            print(f"  -> Cluster {cluster_id} ignoré: trop peu de points ({n_points} < 3).")
            continue

        # Ajuster dynamiquement les paramètres en fonction de la taille du cluster
        dynamic_min_cluster_size = min(min_cluster_size, max(3, n_points // 10))
        dynamic_min_samples      = min(min_samples,      max(2, n_points // 20))

        if n_points < dynamic_min_cluster_size:
            print(f"  -> Cluster {cluster_id} ignoré: {n_points} < min_cluster_size ({dynamic_min_cluster_size}).")
            continue

        print(f"  -> Paramètres HDBSCAN: min_cluster_size={dynamic_min_cluster_size}, min_samples={dynamic_min_samples}")

        # Récupération des embeddings
        cluster_embeddings = np.stack(cluster_data['embedding'])

        try:
            # Supprimez ou ajustez cluster_selection_epsilon si vous avez tout en bruit.
            hdb = hdbscan.HDBSCAN(
                min_cluster_size=dynamic_min_cluster_size,
                min_samples=dynamic_min_samples,
                metric='euclidean',
                cluster_selection_method='eom',
                allow_single_cluster=True
            )
            
            local_hdb_labels = hdb.fit_predict(cluster_embeddings)

            # Statistiques pour ce cluster
            n_clusters_found = len(np.unique(local_hdb_labels[local_hdb_labels != -1]))
            n_points_clustered_local = np.sum(local_hdb_labels != -1)
            print(f"  -> Sous-clusters trouvés par HDBSCAN : {n_clusters_found}")
            print(f"  -> Points classifiés (hors bruit)    : {n_points_clustered_local}")

            # Mettre à jour compteurs globaux
            total_clusters_found     += n_clusters_found
            total_points_clustered   += n_points_clustered_local

            # Décalage des labels locaux (pour éviter collisions entre clusters)
            if n_clusters_found > 0:
                valid_mask = (local_hdb_labels != -1)
                local_hdb_labels[valid_mask] += global_offset
                global_offset = local_hdb_labels.max() + 1

            # Injection des labels HDBSCAN dans hdbscan_labels (index globaux)
            hdbscan_labels[cluster_data.index] = local_hdb_labels

        except Exception as e:
            print(f"Erreur lors du traitement du cluster KMeans {cluster_id}: {e}")
            continue

    print("\n=== Résumé HDBSCAN ===")
    print(f"Total sous-clusters trouvés : {total_clusters_found}")
    print(f"Total points classifiés     : {total_points_clustered}")
    if len(df) > 0:
        pct = (total_points_clustered / len(df)) * 100
        print(f"Pourcentage de points hors bruit: {pct:.2f}%")

    # Assignation finale à la DataFrame
    df['hdbscan_cluster'] = hdbscan_labels
    return df

def add_kmeans_centroids(df, kmeans):
    """
    Ajoute une colonne 'kmeans_centroid' contenant le centroïde KMeans
    de chaque cluster KMeans. Ainsi, on peut calculer la distance d'un
    point à ce centroïde.
    """
    print("\nCalcul du centroïde KMeans pour chaque point...")

    # Récupération de tous les centroïdes KMeans
    centers = kmeans.cluster_centers_

    # On mappe kmeans_cluster -> vecteur centroïde
    df['kmeans_centroid'] = df['kmeans_cluster'].apply(lambda c: centers[c])
    
    print("Centroïdes KMeans ajoutés.")
    return df

def add_ranking(df):
    """
    Calcule la distance de chaque point à son centroïde KMeans,
    puis attribue un rang 'rank' à l'intérieur de chaque sous-cluster HDBSCAN.
    Les points de bruit (hdbscan_cluster = -1) ont distance = inf et rank = NaN.
    """
    print("\nCalcul des distances au centroïde KMeans et ajout du ranking...")

    def distance_to_kmeans_centroid(row):
        if row['hdbscan_cluster'] == -1:
            return float('inf')
        # Distance par rapport au centroïde KMeans (déjà stocké dans 'kmeans_centroid')
        return np.linalg.norm(row['embedding'] - row['kmeans_centroid'])

    # Distance au centroïde KMeans
    df['distance_to_centroid'] = df.apply(distance_to_kmeans_centroid, axis=1)

    # Calcul du rang par (kmeans_cluster, hdbscan_cluster)
    df['rank'] = df[df['hdbscan_cluster'] != -1].groupby(
        ['kmeans_cluster', 'hdbscan_cluster']
    )['distance_to_centroid'].rank(method='first')

    print("Ranking ajouté.")
    return df

def clustering_pipeline_with_hdbscan(df, n_clusters=150, min_cluster_size=5, min_samples=3):
    """
    Pipeline principal:
      1. KMeans
      2. HDBSCAN par cluster KMeans
      3. Ajout centroïdes KMeans
      4. Ranking par distance au centroïde KMeans à l'intérieur des sous-clusters
    """
    print("\n=== Début du pipeline de clustering ===")
    print(f"Dimensions du dataset initial : {df.shape}")

    # On enlève éventuellement les lignes avec embedding manquant
    df = df.dropna(subset=['embedding'])
    if df.empty:
        print("Erreur: Dataset vide après nettoyage.")
        return df, None

    print(f"Dimensions après nettoyage : {df.shape}")

    embeddings = np.stack(df['embedding'].values)

    # S'il y a moins de points que de clusters demandés
    if len(embeddings) < n_clusters:
        n_clusters = max(2, len(embeddings) // 10)
        print(f"Ajustement du nombre de clusters KMeans à : {n_clusters}")

    # 1. KMeans
    kmeans_labels, kmeans_model = apply_kmeans(embeddings, n_clusters=n_clusters)
    df['kmeans_cluster'] = kmeans_labels

    # 2. HDBSCAN dans chaque cluster KMeans
    df = refine_with_hdbscan_per_kmeans_cluster(df, min_cluster_size=min_cluster_size, min_samples=min_samples)

    # [DEBUG] Vérifions les labels tout de suite après la phase HDBSCAN
    print("\n[DEBUG] Distribution hdbscan_cluster (après refine_with_hdbscan_per_kmeans_cluster) :")
    print(df['hdbscan_cluster'].value_counts(dropna=False))

    # 3. Ajouter le centroïde KMeans pour chaque ligne
    df = add_kmeans_centroids(df, kmeans_model)

    # [DEBUG] Vérifions que nous n'avons pas perdu les labels
    print("\n[DEBUG] Distribution hdbscan_cluster (juste après add_kmeans_centroids) :")
    print(df['hdbscan_cluster'].value_counts(dropna=False))

    # 4. Calculer la distance à ce centroïde KMeans et rajouter un rang
    df = add_ranking(df)

    # [DEBUG] Vérifions que nous n'avons pas perdu les labels encore
    print("\n[DEBUG] Distribution hdbscan_cluster (juste après add_ranking) :")
    print(df['hdbscan_cluster'].value_counts(dropna=False))

    print("=== Fin du pipeline de clustering ===\n")
    return df, kmeans_model

def process_full_dataset(file_path, 
                        n_clusters=150, 
                        min_cluster_size=5, 
                        min_samples=3, 
                        sample_frac=None):
    """
    Fonction globale: 
      - charge le dataset 
      - lance le pipeline 
      - sauvegarde le résultat dans un CSV 
      - affiche quelques stats finales
    """
    print("=== Démarrage du traitement global ===")

    # 1. Chargement des données
    df = load_full_dataset(file_path, use_columns=['embedding', 'Message-ID', 'final_body', 'From'], sample_frac=sample_frac)
    if df.empty:
        print("Erreur: dataset vide, arrêt.")
        return

    # 2. Pipeline KMeans + HDBSCAN
    df_with_ranking, kmeans_model = clustering_pipeline_with_hdbscan(
        df,
        n_clusters=n_clusters,
        min_cluster_size=min_cluster_size,
        min_samples=min_samples
    )

    # 3. Sauvegarde du résultat
    if df_with_ranking is not None and not df_with_ranking.empty:
        output_file = "final_dataset_with_hdbscan_ranking_PERMISSIF_1.csv"
        df_with_ranking.to_csv(output_file, index=False)
        print(f"Dataset final sauvegardé dans : {output_file}")

        # 4. Statistiques finales
        unique_clusters = df_with_ranking['hdbscan_cluster'].unique()
        # Nombre de clusters hors bruit
        total_clusters = len(unique_clusters) - (1 if -1 in unique_clusters else 0)
        noise_points   = (df_with_ranking['hdbscan_cluster'] == -1).sum()
        noise_pct      = (noise_points / len(df_with_ranking)) * 100

        print("\n=== Statistiques finales ===")
        print(f"Nombre total de sous-clusters HDBSCAN : {total_clusters}")
        print(f"Points de bruit : {noise_points} ({noise_pct:.2f}%)")
        print(f"Points classifiés : {len(df_with_ranking) - noise_points} ({100 - noise_pct:.2f}%)")
    else:
        print("Aucun résultat à sauvegarder, dataset vide après clustering.")

    print("=== Fin du traitement global ===\n")
'''

In [ ]:
'''if __name__ == "__main__":
    # Exemple d'utilisation
    file_path = '/Users/armandbidault/Desktop/ISD/Data_Science/Clustering/Spam/NoSpam_emails.csv'
    process_full_dataset(
        file_path,
        n_clusters=150,          # Nombre de clusters KMeans
        min_cluster_size=30,      # Taille minimum d'un cluster HDBSCAN
        min_samples=2,
                   # Nombre minimum d'échantillons pour HDBSCAN
    
    )'''

=== Démarrage du traitement global ===
Chargement des données...
Chargement terminé. Nombre de lignes: 363684

=== Début du pipeline de clustering ===
Dimensions du dataset initial : (363684, 4)
Dimensions après nettoyage : (363684, 4)
Application de KMeans...
Nombre de points: 363684
KMeans terminé. Distribution des clusters :
 - Cluster KMeans 0: 1522 points
 - Cluster KMeans 1: 2487 points
 - Cluster KMeans 2: 3759 points
 - Cluster KMeans 3: 4104 points
 - Cluster KMeans 4: 2979 points
 - Cluster KMeans 5: 2589 points
 - Cluster KMeans 6: 1317 points
 - Cluster KMeans 7: 2631 points
 - Cluster KMeans 8: 1713 points
 - Cluster KMeans 9: 2879 points
 - Cluster KMeans 10: 2249 points
 - Cluster KMeans 11: 2371 points
 - Cluster KMeans 12: 4581 points
 - Cluster KMeans 13: 2822 points
 - Cluster KMeans 14: 796 points
 - Cluster KMeans 15: 3616 points
 - Cluster KMeans 16: 2872 points
 - Cluster KMeans 17: 2497 points
 - Cluster KMeans 18: 2954 points
 - Cluster KMeans 19: 2333 points
 

Clusters KMeans:   0%|          | 0/150 [00:00<?, ?it/s]


Traitement du cluster KMeans 107 contenant 2818 points.
  -> Paramètres HDBSCAN: min_cluster_size=30, min_samples=2


Clusters KMeans:   1%|          | 1/150 [00:04<09:58,  4.02s/it]

  -> Sous-clusters trouvés par HDBSCAN : 1
  -> Points classifiés (hors bruit)    : 1981

Traitement du cluster KMeans 67 contenant 4402 points.
  -> Paramètres HDBSCAN: min_cluster_size=30, min_samples=2


...

Clusters KMeans:  99%|█████████▊| 148/150 [08:22<00:03,  1.83s/it]

  -> Sous-clusters trouvés par HDBSCAN : 6
  -> Points classifiés (hors bruit)    : 278

Traitement du cluster KMeans 32 contenant 1175 points.
  -> Paramètres HDBSCAN: min_cluster_size=30, min_samples=2


Clusters KMeans: 100%|██████████| 150/150 [08:23<00:00,  3.36s/it]

  -> Sous-clusters trouvés par HDBSCAN : 4
  -> Points classifiés (hors bruit)    : 1167

Traitement du cluster KMeans 99 contenant 282 points.
  -> Paramètres HDBSCAN: min_cluster_size=28, min_samples=2
  -> Sous-clusters trouvés par HDBSCAN : 4
  -> Points classifiés (hors bruit)    : 268

=== Résumé HDBSCAN ===
Total sous-clusters trouvés : 562
Total points classifiés     : 163988
Pourcentage de points hors bruit: 45.09%

[DEBUG] Distribution hdbscan_cluster (après refine_with_hdbscan_per_kmeans_cluster) :
hdbscan_cluster
-1      199696
 125      4235
 309      3584
 406      3432
 40       3174
         ...  
 93         30
 295        30
 153        30
 458        30
 449        30
Name: count, Length: 563, dtype: int64

Calcul du centroïde KMeans pour chaque point...
Centroïdes KMeans ajoutés.

[DEBUG] Distribution hdbscan_cluster (juste après add_kmeans_centroids) :
hdbscan_cluster
-1      199696
 125      4235
 309      3584
 406      3432
 40       3174
         ...  
 93     

Ranking ajouté.

[DEBUG] Distribution hdbscan_cluster (juste après add_ranking) :
hdbscan_cluster
-1      199696
 125      4235
 309      3584
 406      3432
 40       3174
         ...  
 93         30
 295        30
 153        30
 458        30
 449        30
Name: count, Length: 563, dtype: int64
=== Fin du pipeline de clustering ===

Dataset final sauvegardé dans : final_dataset_with_hdbscan_ranking_PERMISSIF_1.csv

=== Statistiques finales ===
Nombre total de sous-clusters HDBSCAN : 562
Points de bruit : 199696 (54.91%)
Points classifiés : 163988 (45.09%)
=== Fin du traitement global ===



# Clustering

In [3]:
data = pd.read_parquet('NoSpam_emails.parquet')

In [4]:
# Séparer les doublons dans un nouveau DataFrame
data_duplicates = data[data.duplicated(subset=['final_body'], keep='first')]

# Supprimer les doublons dans le DataFrame principal
data.drop_duplicates(subset=['final_body'], keep='first', inplace=True)

# Affichage des résultats
print(data.shape)
print(data_duplicates.shape)

(156997, 10)
(206687, 10)


## Kmean

In [5]:
import ast
# Étape 1 : Convertir la colonne d'embeddings en tableaux numpy
data['embedding'] = data['embedding'].apply(
    lambda x: np.array(ast.literal_eval(x)) if isinstance(x, str) else np.array(x)
)

# Vérifier que toutes les dimensions sont cohérentes
embedding_matrix = np.stack(data['embedding'].values)

# Étape 2 : Appliquer KMeans
n_clusters = 2000  # Par exemple, choisir 2 clusters
kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init='auto')
labels = kmeans.fit_predict(embedding_matrix)

# Étape 3 : Ajouter les labels au DataFrame
data['KMeans_Labels'] = labels

# Afficher le DataFrame mis à jour
data

,file,message,parsed_email,Message-ID,X-FileName,Body,final_body,embedding,spam,From,KMeans_Labels
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...,{'Message-ID': '<18782981.1075855378110.JavaMa...,<18782981.1075855378110.JavaMail.evans@thyme>,pallen (Non-Privileged).pst,\nHere is our forecast\n\n,forecast,"[-0.2228, -0.1683, 0.414, -0.1859, 0.7666, -0....",0,phillip.allen@enron.com,577
1,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...,{'Message-ID': '<13505866.1075863688222.JavaMa...,<13505866.1075863688222.JavaMail.evans@thyme>,pallen.nsf,"\nRandy,\n\n Can you send me a schedule of the...","Randy, send schedule salary level everyone sch...","[0.4868, 0.3755, -0.836, -0.723, 1.365, -0.114...",0,phillip.allen@enron.com,351
2,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...,{'Message-ID': '<30922949.1075863688243.JavaMa...,<30922949.1075863688243.JavaMail.evans@thyme>,pallen.nsf,\nLet's shoot for Tuesday at 11:45.,Let's shoot Tuesday 11:45.,"[-0.03079, 0.02507, -0.4038, -0.3984, 1.139, 0...",0,phillip.allen@enron.com,765
3,allen-p/_sent_mail/1002.,Message-ID: <30965995.1075863688265.JavaMail.e...,{'Message-ID': '<30965995.1075863688265.JavaMa...,<30965995.1075863688265.JavaMail.evans@thyme>,pallen.nsf,"\nGreg,\n\n How about either next Tuesday or T...","Greg, either next Tuesday Thursday? Phillip","[-0.4834, 0.2832, -1.531, -0.6226, 0.0867, 0.4...",0,phillip.allen@enron.com,765
4,allen-p/_sent_mail/1003.,Message-ID: <16254169.1075863688286.JavaMail.e...,{'Message-ID': '<16254169.1075863688286.JavaMa...,<16254169.1075863688286.JavaMail.evans@thyme>,pallen.nsf,\nPlease cc the following distribution list wi...,Please cc following distribution list updates:...,"[0.2007, -0.7554, -0.322, -0.1583, 0.454, -0.4...",0,phillip.allen@enron.com,596
...,...,...,...,...,...,...,...,...,...,...,...
434014,zufferli-j/sent_items/89.,Message-ID: <24358278.1075842029773.JavaMail.e...,{'Message-ID': '<24358278.1075842029773.JavaMa...,<24358278.1075842029773.JavaMail.evans@thyme>,john zufferli 6-26-02.PST,\n\nEnron is willing to perform the operation ...,Enron willing perform operation question (1) (...,"[0.00992, -0.2651, -0.605, 0.1359, 1.509, 0.04...",0,john.zufferli@enron.com,1126
434015,zufferli-j/sent_items/91.,Message-ID: <23829224.1075842029820.JavaMail.e...,{'Message-ID': '<23829224.1075842029820.JavaMa...,<23829224.1075842029820.JavaMail.evans@thyme>,john zufferli 6-26-02.PST,"\nNicole and everyone, I think the directory i...","Nicole everyone, think directory i:\canadian\e...","[1.218, -0.943, 0.1938, 1.032, 0.4238, -0.3433...",0,john.zufferli@enron.com,999
434016,zufferli-j/sent_items/95.,Message-ID: <26807948.1075842029936.JavaMail.e...,{'Message-ID': '<26807948.1075842029936.JavaMa...,<26807948.1075842029936.JavaMail.evans@thyme>,john zufferli 6-26-02.PST,\nThis is a trade with OIL-SPEC-HEDGE-NG (John...,trade OIL-SPEC-HEDGE-NG (John Lavorato's book)...,"[0.2764, -0.03079, -0.6875, 0.05423, -1.289, 0...",0,john.zufferli@enron.com,1332
434017,zufferli-j/sent_items/96.,Message-ID: <25835861.1075842029959.JavaMail.e...,{'Message-ID': '<25835861.1075842029959.JavaMa...,<25835861.1075842029959.JavaMail.evans@thyme>,john zufferli 6-26-02.PST,\nSome of my position is with the Alberta Term...,"position Alberta Term book, send positions dir...","[0.7695, -0.5005, -1.423, -0.1415, -0.4844, 0....",0,john.zufferli@enron.com,569


In [6]:
data['KMeans_Labels'].value_counts()

KMeans_Labels
1386    563
1797    420
1905    352
821     333
622     309
       ... 
1027      5
1337      4
1725      4
1307      3
1871      2
Name: count, Length: 2000, dtype: int64

## HDBSCAN

In [7]:
df = data[['KMeans_Labels', 'embedding']]
# Étape 1 : Calculer les centroïdes des clusters KMeans
def calculate_centroid(embeddings):
    return np.mean(np.vstack(embeddings), axis=0)

# Grouper par `kmeans_cluster` et calculer les centroïdes
centroids_df = (
    df.groupby("KMeans_Labels")["embedding"]
    .apply(calculate_centroid)
    .reset_index()
)

# Renommer la colonne pour plus de clarté
centroids_df.rename(columns={"embedding": "centroid"}, inplace=True)

# Étape 2 : Appliquer HDBSCAN sur les centroïdes
# Convertir les centroïdes en une matrice numpy
centroids_matrix = np.vstack(centroids_df["centroid"])

# Appliquer HDBSCAN
hdbscan_model = hdbscan.HDBSCAN(min_cluster_size=2, min_samples=1)
hdbscan_labels = hdbscan_model.fit_predict(centroids_matrix)

# Ajouter les labels HDBSCAN au DataFrame des centroïdes
centroids_df["hdbscan_cluster"] = hdbscan_labels

centroids_df["hdbscan_cluster"].value_counts()

hdbscan_cluster
-1      1022
 2        17
 157      13
 31       10
 202      10
        ... 
 220       2
 312       2
 251       2
 72        2
 41        2
Name: count, Length: 329, dtype: int64

In [8]:
# Étape 3 : Associer les clusters HDBSCAN aux clusters KMeans initiaux
# Faire une jointure avec le DataFrame original
merged_df = data.merge(centroids_df[['KMeans_Labels','hdbscan_cluster']], on="KMeans_Labels", how='left')

merged_df

,file,message,parsed_email,Message-ID,X-FileName,Body,final_body,embedding,spam,From,KMeans_Labels,hdbscan_cluster
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...,{'Message-ID': '<18782981.1075855378110.JavaMa...,<18782981.1075855378110.JavaMail.evans@thyme>,pallen (Non-Privileged).pst,\nHere is our forecast\n\n,forecast,"[-0.2228, -0.1683, 0.414, -0.1859, 0.7666, -0....",0,phillip.allen@enron.com,577,-1
1,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...,{'Message-ID': '<13505866.1075863688222.JavaMa...,<13505866.1075863688222.JavaMail.evans@thyme>,pallen.nsf,"\nRandy,\n\n Can you send me a schedule of the...","Randy, send schedule salary level everyone sch...","[0.4868, 0.3755, -0.836, -0.723, 1.365, -0.114...",0,phillip.allen@enron.com,351,150
2,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...,{'Message-ID': '<30922949.1075863688243.JavaMa...,<30922949.1075863688243.JavaMail.evans@thyme>,pallen.nsf,\nLet's shoot for Tuesday at 11:45.,Let's shoot Tuesday 11:45.,"[-0.03079, 0.02507, -0.4038, -0.3984, 1.139, 0...",0,phillip.allen@enron.com,765,89
3,allen-p/_sent_mail/1002.,Message-ID: <30965995.1075863688265.JavaMail.e...,{'Message-ID': '<30965995.1075863688265.JavaMa...,<30965995.1075863688265.JavaMail.evans@thyme>,pallen.nsf,"\nGreg,\n\n How about either next Tuesday or T...","Greg, either next Tuesday Thursday? Phillip","[-0.4834, 0.2832, -1.531, -0.6226, 0.0867, 0.4...",0,phillip.allen@enron.com,765,89
4,allen-p/_sent_mail/1003.,Message-ID: <16254169.1075863688286.JavaMail.e...,{'Message-ID': '<16254169.1075863688286.JavaMa...,<16254169.1075863688286.JavaMail.evans@thyme>,pallen.nsf,\nPlease cc the following distribution list wi...,Please cc following distribution list updates:...,"[0.2007, -0.7554, -0.322, -0.1583, 0.454, -0.4...",0,phillip.allen@enron.com,596,-1
...,...,...,...,...,...,...,...,...,...,...,...,...
156992,zufferli-j/sent_items/89.,Message-ID: <24358278.1075842029773.JavaMail.e...,{'Message-ID': '<24358278.1075842029773.JavaMa...,<24358278.1075842029773.JavaMail.evans@thyme>,john zufferli 6-26-02.PST,\n\nEnron is willing to perform the operation ...,Enron willing perform operation question (1) (...,"[0.00992, -0.2651, -0.605, 0.1359, 1.509, 0.04...",0,john.zufferli@enron.com,1126,-1
156993,zufferli-j/sent_items/91.,Message-ID: <23829224.1075842029820.JavaMail.e...,{'Message-ID': '<23829224.1075842029820.JavaMa...,<23829224.1075842029820.JavaMail.evans@thyme>,john zufferli 6-26-02.PST,"\nNicole and everyone, I think the directory i...","Nicole everyone, think directory i:\canadian\e...","[1.218, -0.943, 0.1938, 1.032, 0.4238, -0.3433...",0,john.zufferli@enron.com,999,157
156994,zufferli-j/sent_items/95.,Message-ID: <26807948.1075842029936.JavaMail.e...,{'Message-ID': '<26807948.1075842029936.JavaMa...,<26807948.1075842029936.JavaMail.evans@thyme>,john zufferli 6-26-02.PST,\nThis is a trade with OIL-SPEC-HEDGE-NG (John...,trade OIL-SPEC-HEDGE-NG (John Lavorato's book)...,"[0.2764, -0.03079, -0.6875, 0.05423, -1.289, 0...",0,john.zufferli@enron.com,1332,66
156995,zufferli-j/sent_items/96.,Message-ID: <25835861.1075842029959.JavaMail.e...,{'Message-ID': '<25835861.1075842029959.JavaMa...,<25835861.1075842029959.JavaMail.evans@thyme>,john zufferli 6-26-02.PST,\nSome of my position is with the Alberta Term...,"position Alberta Term book, send positions dir...","[0.7695, -0.5005, -1.423, -0.1415, -0.4844, 0....",0,john.zufferli@enron.com,569,256


In [9]:
merged_df.to_parquet('KMeans_HDBSCAN_Clustering.parquet', index=False)
'''merged_df = pd.read_csv('KMeans_HDBSCAN_Clustering.csv')'''

"merged_df = pd.read_csv('KMeans_HDBSCAN_Clustering.csv')"

In [18]:
data_cluster = merged_df[merged_df['hdbscan_cluster'] != -1]
data_cluster

,file,message,parsed_email,Message-ID,X-FileName,Body,final_body,embedding,spam,From,KMeans_Labels,hdbscan_cluster
1,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...,{'Message-ID': '<13505866.1075863688222.JavaMa...,<13505866.1075863688222.JavaMail.evans@thyme>,pallen.nsf,"\nRandy,\n\n Can you send me a schedule of the...","Randy, send schedule salary level everyone sch...","[0.4868, 0.3755, -0.836, -0.723, 1.365, -0.114...",0,phillip.allen@enron.com,351,150
2,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...,{'Message-ID': '<30922949.1075863688243.JavaMa...,<30922949.1075863688243.JavaMail.evans@thyme>,pallen.nsf,\nLet's shoot for Tuesday at 11:45.,Let's shoot Tuesday 11:45.,"[-0.03079, 0.02507, -0.4038, -0.3984, 1.139, 0...",0,phillip.allen@enron.com,765,89
3,allen-p/_sent_mail/1002.,Message-ID: <30965995.1075863688265.JavaMail.e...,{'Message-ID': '<30965995.1075863688265.JavaMa...,<30965995.1075863688265.JavaMail.evans@thyme>,pallen.nsf,"\nGreg,\n\n How about either next Tuesday or T...","Greg, either next Tuesday Thursday? Phillip","[-0.4834, 0.2832, -1.531, -0.6226, 0.0867, 0.4...",0,phillip.allen@enron.com,765,89
6,allen-p/_sent_mail/101.,Message-ID: <20641191.1075855687472.JavaMail.e...,{'Message-ID': '<20641191.1075855687472.JavaMa...,<20641191.1075855687472.JavaMail.evans@thyme>,pallen.nsf,\n1. login: pallen pw: ke9davis\n\n I don't t...,1. login: pallen pw: ke9davis think required I...,"[0.2861, 0.54, -1.153, -0.2043, 0.3127, -0.184...",0,phillip.allen@enron.com,840,139
7,allen-p/_sent_mail/102.,Message-ID: <30795301.1075855687494.JavaMail.e...,{'Message-ID': '<30795301.1075855687494.JavaMa...,<30795301.1075855687494.JavaMail.evans@thyme>,pallen.nsf,\n---------------------- Forwarded by Phillip ...,Subject: FW: fixed forward Collar floor gas pr...,"[0.274, -0.598, -0.03268, 0.6514, -0.4514, -0....",0,phillip.allen@enron.com,1229,221
...,...,...,...,...,...,...,...,...,...,...,...,...
156990,zufferli-j/sent_items/83.,Message-ID: <24246770.1075842029601.JavaMail.e...,{'Message-ID': '<24246770.1075842029601.JavaMa...,<24246770.1075842029601.JavaMail.evans@thyme>,john zufferli 6-26-02.PST,\n\nIn addition to a list of software and syst...,addition list software systems compiled th= e ...,"[0.4297, -1.052, 0.0597, -0.4756, 0.477, 0.512...",0,john.zufferli@enron.com,373,271
156991,zufferli-j/sent_items/84.,Message-ID: <30449512.1075842029649.JavaMail.e...,{'Message-ID': '<30449512.1075842029649.JavaMa...,<30449512.1075842029649.JavaMail.evans@thyme>,john zufferli 6-26-02.PST,\nPlease set up access for the digital certifi...,Please set access digital certificate Enron TA...,"[-0.7466, -0.1177, -0.7124, -0.559, 0.489, -0....",0,john.zufferli@enron.com,1372,314
156993,zufferli-j/sent_items/91.,Message-ID: <23829224.1075842029820.JavaMail.e...,{'Message-ID': '<23829224.1075842029820.JavaMa...,<23829224.1075842029820.JavaMail.evans@thyme>,john zufferli 6-26-02.PST,"\nNicole and everyone, I think the directory i...","Nicole everyone, think directory i:\canadian\e...","[1.218, -0.943, 0.1938, 1.032, 0.4238, -0.3433...",0,john.zufferli@enron.com,999,157
156994,zufferli-j/sent_items/95.,Message-ID: <26807948.1075842029936.JavaMail.e...,{'Message-ID': '<26807948.1075842029936.JavaMa...,<26807948.1075842029936.JavaMail.evans@thyme>,john zufferli 6-26-02.PST,\nThis is a trade with OIL-SPEC-HEDGE-NG (John...,trade OIL-SPEC-HEDGE-NG (John Lavorato's book)...,"[0.2764, -0.03079, -0.6875, 0.05423, -1.289, 0...",0,john.zufferli@enron.com,1332,66


In [ ]:
data_cluster[data_cluster['KMeans_Labels']==1]

: 

In [36]:
import pandas as pd
import numpy as np
from tqdm import tqdm

def filter_top_k_elements(data_cluster, k=20):
    """
    Filtre le DataFrame pour ne garder que les `k` éléments les plus proches des centroïdes pour chaque cluster.
    
    Paramètres :
    - data_cluster : DataFrame contenant les colonnes 'Kmeans_Labels' (cluster) et 'Embedding' (vecteurs).
    - k : Nombre d'éléments à sélectionner par cluster (par défaut : 10).
    
    Retour :
    - DataFrame filtré avec les `k` éléments les plus proches des centroïdes pour chaque cluster.
    """
    tqdm.pandas(desc="Calcul des centroïdes")

    # Étape 1 : Calcul des centroïdes pour chaque cluster
    centroids = (
        data_cluster.groupby('hdbscan_cluster')['embedding']
        .progress_apply(lambda x: np.mean(np.stack(x), axis=0))
        .to_dict()
    )
    print(len(data_cluster['hdbscan_cluster'].unique()))
    print(f"Centroïdes calculés pour {len(centroids)} clusters.")
    # Étape 2 : Calcul des distances aux centroïdes
    def calculate_distance(row):
        cluster = row['hdbscan_cluster']
        centroid = centroids[cluster]
        embedding = np.array(row['embedding'])
        return np.linalg.norm(embedding - centroid)

    tqdm.pandas(desc="Calcul des distances aux centroïdes")
    data_cluster['Distance_to_Centroid'] = data_cluster.progress_apply(calculate_distance, axis=1)

    # Étape 3 : Sélection des `k` éléments les plus proches pour chaque cluster
    tqdm.pandas(desc="Sélection des top éléments par cluster")
    top_k_per_cluster = (
        data_cluster.groupby('hdbscan_cluster', group_keys=False)
        .progress_apply(lambda x: x.nsmallest(k, 'Distance_to_Centroid'))
    )

    return top_k_per_cluster


In [37]:
filtered_data_cluster_HDBSCAN = filter_top_k_elements(data_cluster, k=20)
print(filtered_data_cluster_HDBSCAN)


Calcul des centroïdes: 100%|██████████| 328/328 [00:00<00:00, 2013.81it/s]


328
Centroïdes calculés pour 328 clusters.


Calcul des distances aux centroïdes: 100%|██████████| 93106/93106 [00:00<00:00, 188243.57it/s]
/var/folders/x6/w927_g5j2lsf_r6g6lw8cjzw0000gn/T/ipykernel_25491/1665816055.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_cluster['Distance_to_Centroid'] = data_cluster.progress_apply(calculate_distance, axis=1)
Sélection des top éléments par cluster: 100%|██████████| 328/328 [00:00<00:00, 3030.33it/s]

                                     file  \
98614         meyers-a/deleted_items/351.   
98618         meyers-a/deleted_items/355.   
98617         meyers-a/deleted_items/354.   
98704         meyers-a/deleted_items/436.   
98605         meyers-a/deleted_items/343.   
...                                   ...   
7328           beck-s/all_documents/2511.   
56518          hyvl-d/all_documents/1524.   
92931          mann-k/all_documents/2216.   
121988  shackleton-s/all_documents/10112.   
113526        sager-e/all_documents/1121.   

                                                  message  \
98614   Message-ID: <15489782.1075841297698.JavaMail.e...   
98618   Message-ID: <27677080.1075841297796.JavaMail.e...   
98617   Message-ID: <27053622.1075841297772.JavaMail.e...   
98704   Message-ID: <19044266.1075841299880.JavaMail.e...   
98605   Message-ID: <19500157.1075841297502.JavaMail.e...   
...                                                   ...   
7328    Message-ID: <4617246.107

In [38]:
filtered_data_cluster_HDBSCAN

,file,message,parsed_email,Message-ID,X-FileName,Body,final_body,embedding,spam,From,KMeans_Labels,hdbscan_cluster,Distance_to_Centroid
98614,meyers-a/deleted_items/351.,Message-ID: <15489782.1075841297698.JavaMail.e...,{'Message-ID': '<15489782.1075841297698.JavaMa...,<15489782.1075841297698.JavaMail.evans@thyme>,bert meyers 6-25-02.PST,\n\n\nStart Date: 1/24/02; HourAhead hour: 13;...,Start Date: 1/24/02; HourAhead hour: 13; HourA...,"[-0.06107, -0.551, -0.5913, -1.205, 1.005, -0....",0,pete.davis@enron.com,1120,0,4.384933
98618,meyers-a/deleted_items/355.,Message-ID: <27677080.1075841297796.JavaMail.e...,{'Message-ID': '<27677080.1075841297796.JavaMa...,<27677080.1075841297796.JavaMail.evans@thyme>,bert meyers 6-25-02.PST,\n\n\nStart Date: 1/24/02; HourAhead hour: 10;...,Start Date: 1/24/02; HourAhead hour: 10; HourA...,"[-0.0902, -0.4102, -0.4253, -1.234, 0.9966, -0...",0,pete.davis@enron.com,1120,0,4.489759
98617,meyers-a/deleted_items/354.,Message-ID: <27053622.1075841297772.JavaMail.e...,{'Message-ID': '<27053622.1075841297772.JavaMa...,<27053622.1075841297772.JavaMail.evans@thyme>,bert meyers 6-25-02.PST,\n\n\nStart Date: 1/24/02; HourAhead hour: 11;...,Start Date: 1/24/02; HourAhead hour: 11; HourA...,"[-0.09845, -0.5376, -0.4358, -1.298, 1.08, -0....",0,pete.davis@enron.com,1120,0,4.522798
98704,meyers-a/deleted_items/436.,Message-ID: <19044266.1075841299880.JavaMail.e...,{'Message-ID': '<19044266.1075841299880.JavaMa...,<19044266.1075841299880.JavaMail.evans@thyme>,bert meyers 6-25-02.PST,\n\n\nStart Date: 1/21/02; HourAhead hour: 12;...,Start Date: 1/21/02; HourAhead hour: 12; HourA...,"[-0.132, -0.5293, -0.595, -0.97, 0.6724, -0.05...",0,pete.davis@enron.com,1120,0,4.532666
98605,meyers-a/deleted_items/343.,Message-ID: <19500157.1075841297502.JavaMail.e...,{'Message-ID': '<19500157.1075841297502.JavaMa...,<19500157.1075841297502.JavaMail.evans@thyme>,bert meyers 6-25-02.PST,\n\n\nStart Date: 1/24/02; HourAhead hour: 19;...,Start Date: 1/24/02; HourAhead hour: 19; HourA...,"[-0.054, -0.6216, -0.457, -1.314, 0.759, -0.30...",0,pete.davis@enron.com,1120,0,4.589929
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7328,beck-s/all_documents/2511.,Message-ID: <4617246.1075855806069.JavaMail.ev...,{'Message-ID': '<4617246.1075855806069.JavaMai...,<4617246.1075855806069.JavaMail.evans@thyme>,sbeck.nsf,\nI agree with the section written for settlem...,"Subject: Interim Operational, Accounting Tax S...","[-0.892, -0.864, 0.2305, -1.005, 0.2346, -0.03...",0,bob.klein@enron.com,1082,327,11.432283
56518,hyvl-d/all_documents/1524.,Message-ID: <16818862.1075842248023.JavaMail.e...,{'Message-ID': '<16818862.1075842248023.JavaMa...,<16818862.1075842248023.JavaMail.evans@thyme>,dhyvl.nsf,\nHere is a draft that does not address the En...,draft address Enron Services deal operating ag...,"[-0.836, -0.496, -0.1189, -0.53, 1.049, -0.076...",0,chris.foster@enron.com,1082,327,11.449037
92931,mann-k/all_documents/2216.,Message-ID: <20798891.1075845629294.JavaMail.e...,{'Message-ID': '<20798891.1075845629294.JavaMa...,<20798891.1075845629294.JavaMail.evans@thyme>,kmann.nsf,\nBen Jacoby has asked that I forward the atta...,Ben Jacoby asked forward attached drafts lette...,"[-0.7876, -0.6455, -0.2404, 0.2974, 1.566, 0.0...",0,kay.mann@enron.com,1082,327,11.478019
121988,shackleton-s/all_documents/10112.,Message-ID: <22610625.1075844720790.JavaMail.e...,{'Message-ID': '<22610625.1075844720790.JavaMa...,<22610625.1075844720790.JavaMail.evans@thyme>,sshackle.nsf,"\nCheryl,\n\nPlease add the RBC Dominion Secur...","Subject: Update List Cheryl, Please add RBC Do...","[-0.278, -1.009, 0.1251, -0.946, 0.3445, 0.225...",0,aneela.charania@enron.com,1082,327,11.494771


In [39]:
filtered_data_cluster_HDBSCAN.to_parquet('Top20_Centroide_HDBSCAN.parquet', index=False)

In [5]:
data_cluster_bruit = merged_df[merged_df['hdbscan_cluster'] == -1]
data_cluster_bruit.to_csv('Cluster_KMeans_HDBSCAN_noiseTRUE.csv', index=False)

In [6]:
def display_random_final_body(dataframe, n=10, seed=None):
    """
    Affiche de manière aléatoire des textes de la colonne 'final_body'.

    Params:
        dataframe (pd.DataFrame): Le DataFrame contenant la colonne 'final_body'.
        n (int): Nombre de textes à afficher. Par défaut : 10.
        seed (int): Graine pour la reproductibilité du tirage aléatoire. Par défaut : None.
    """
    # Ajuster les paramètres pour afficher tout le texte
    pd.set_option('display.max_colwidth', None)

    # Tirer un échantillon aléatoire de la colonne 'final_body'
    sample_texts = dataframe['final_body'].sample(n=n, random_state=seed)

    # Afficher les textes
    for i, text in enumerate(sample_texts, start=1):
        print(f"--- Texte {i} (Index: {sample_texts.index[i-1]}) ---")
        print(text)
        print("\n" + "=" * 50 + "\n")

    # Réinitialiser les options de pandas
    pd.reset_option('display.max_colwidth')

In [7]:
display_random_final_body(data_cluster_bruit)

--- Texte 1 (Index: 35712) ---
Subject: Chevron - Winter monthly nom 413447 right - swing deal 413462 needs extended match term 413447, buyback 413472 Daren J Farmer 10/18/2000 03:48 PM Lee, deal 413447 (Chevron @ Olefins), flex nom 6,000 12,000/day. monthly nom daily nom? swing deal winter? current swing deal Oct only. Oct, pulled average 16,000/day.


--- Texte 2 (Index: 100256) ---
Subject: Keyspan true-up Janet, Colleen Sullivan replaced Kate Fraser team finalize negotiations folks Keyspan. intention finish calculations Colleen requested next days get feel offsets occur proxy pricing given certain decisions. able finish Keyspan. Hopefully, Ed Anderson Melissa Reges taking vacations while. Colleen talked Scott internally releasing East Origination Group's cut accrual profits Year 2 asset management deal. problem end quarter. Sorry delay, Ruth Janet R Dietrich 09/08/2000 12:43 PM Ruth, gotten everything finalized Keyspan re: one-year deal? so, let's finalize internal dollar true-ups 

In [17]:
from sklearn.cluster import AgglomerativeClustering

def perform_ahc_on_text_embeddings(df, embedding_col, n_clusters=5, metric='euclidean', linkage_method='ward'):
    """
    Applique Agglomerative Hierarchical Clustering (AHC) sur des embeddings textuels.

    :param df: pandas.DataFrame contenant une colonne d'embeddings
    :param embedding_col: Nom de la colonne contenant les embeddings
    :param n_clusters: Nombre de clusters à générer
    :param metric: Métrique de distance ('euclidean', 'cosine', etc.)
    :param linkage_method: Méthode de linkage ('ward', 'complete', 'average', 'single')
    :return: DataFrame avec une colonne supplémentaire 'cluster_label'
    """
    # Vérification si la colonne des embeddings existe
    if embedding_col not in df.columns:
        raise ValueError(f"La colonne '{embedding_col}' n'existe pas dans le DataFrame.")

    # Conversion des embeddings en tableau numpy
    embeddings = np.array(df[embedding_col].tolist())

    # Étape 1 : Application d'Agglomerative Clustering
    clustering = AgglomerativeClustering(
        n_clusters=n_clusters,
        metric=metric,
        linkage=linkage_method
    )
    cluster_labels = clustering.fit_predict(embeddings)

    # Ajouter les labels des clusters au DataFrame
    df['cluster_label'] = cluster_labels

    return df


: 

In [ ]:
data_with_clusters = perform_ahc_on_text_embeddings(
    data_cluster_bruit,
    embedding_col='embedding',
    n_clusters=5,         # Par exemple
    metric='euclidean',   # Distance métrique pour les embeddings
    linkage_method='ward' # Méthode de linkage (Ward pour variance minimale)
)

print(data_cluster_bruit.head())
